In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

# 訓練データ取得
mnist_train = MNIST(root='data',
                    train=True,
                    download=True,
                    transform=transforms.ToTensor())

# テストデータ取得
mnist_test = MNIST(root='data',
                   train=False,
                   download=True,
                   transform=transforms.ToTensor())

print('訓練データ数:', len(mnist_train))
print('テストデータ数:', len(mnist_test))

# DataLoaderの設定
img_size = 28
batch_size = 256
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(img_size*img_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, img_size*img_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)

# 最適化手法
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# 損失ログ
train_losses = []
test_losses = []

# 訓練
for i in range(10):
    net.train() # 訓練モード
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):
        x, t = x.to(device), t.to(device)
        y = net(x)
        loss = F.cross_entropy(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    train_losses.append(loss_train)
    
    net.eval() # 評価モード
    loss_test = 0
    for j, (x, t) in enumerate(test_loader):
        x, t = x.to(device), t.to(device)
        y = net(x)
        loss = F.cross_entropy(y, t)
        loss_test += loss.item()
    loss_test /= j+1
    test_losses.append(loss_test)
    
    if i%1 == 0:
        print('Epoch:', i, 'Loss_Train:', loss_train, 'Loss_Test:', loss_test)

# 誤差の推移をグラフ表示
import matplotlib.pyplot as plt

plt.plot(range(len(train_losses)), train_losses, label='train')
plt.plot(range(len(test_losses)), test_losses, label='test')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

# 正解率を算出
correct = 0
total = 0
for i, (x, t) in enumerate(test_loader):
    x, t = x.to(device), t.to(device)
    y = net(x)
    correct += (y.argmax(1) == t).sum().item()
    total += len(x)
print('Accuracy:', str(correct/total*100) + '%')

訓練データ: Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
訓練データ数: 60000
テストデータ数: 10000
